In [24]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.layers import BatchNormalization
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [25]:
# Load the dataset
df_train = pd.read_csv('encoded_train_two.csv', encoding='ISO-8859-1')
df_test = pd.read_csv('encoded_submission_two.csv', encoding='ISO-8859-1')

In [26]:
# Split the data into features and target
X = df_train.drop('is_converted', axis=1)
y = df_train['is_converted']

In [27]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
df_test.drop(columns='id', axis=1, inplace=True)
x_submission = df_test.drop("is_converted", axis=1)
y_submission = df_test["is_converted"]

In [29]:
# # Apply SMOTE
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [30]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)  # Assuming X_val is your validation set features

x_submission_scaled = scaler.transform(x_submission.astype(float))

In [31]:
# # Calculate class weights
# class_weights = compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(y_train),
#     y=y_train)

# class_weight_dict = dict(enumerate(class_weights))

In [32]:
# Define the model with increased complexity, L1 regularization, and Batch Normalization
model = models.Sequential([
    # Input layer with Batch Normalization
    layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],), kernel_regularizer=regularizers.l1(0.0001)),
    BatchNormalization(),
    layers.Dropout(0.5),
    
    # Additional hidden layer with L1 regularization and Batch Normalization
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l1(0.0001)),
    BatchNormalization(),
    layers.Dropout(0.5),
    
    # Another hidden layer
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.0001)),
    BatchNormalization(),
    layers.Dropout(0.5),
    
    # Output layer
    layers.Dense(1, activation='sigmoid')
])


In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
# Train the model with class_weight argument
history = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_val_scaled, y_val), batch_size=32, verbose=1)

Epoch 1/100


1395/1395 [==============================] - 4s 2ms/step - loss: 0.6413 - accuracy: 0.8770 - val_loss: 0.4963 - val_accuracy: 0.9160
Epoch 2/100
1395/1395 [==============================] - 3s 2ms/step - loss: 0.4695 - accuracy: 0.9160 - val_loss: 0.3977 - val_accuracy: 0.9210
Epoch 3/100
1395/1395 [==============================] - 4s 3ms/step - loss: 0.3756 - accuracy: 0.9234 - val_loss: 0.3207 - val_accuracy: 0.9345
Epoch 4/100
1395/1395 [==============================] - 6s 4ms/step - loss: 0.3185 - accuracy: 0.9294 - val_loss: 0.2841 - val_accuracy: 0.9398
Epoch 5/100
1395/1395 [==============================] - 3s 2ms/step - loss: 0.2906 - accuracy: 0.9313 - val_loss: 0.2634 - val_accuracy: 0.9425
Epoch 6/100
1395/1395 [==============================] - 3s 2ms/step - loss: 0.2803 - accuracy: 0.9345 - val_loss: 0.2535 - val_accuracy: 0.9469
Epoch 7/100
1395/1395 [==============================] - 3s 2ms/step - loss: 0.2691 - accuracy: 0.9379 - val_loss: 0.2450 - val_accuracy: 0.94

In [35]:
# Evaluate the model with the same validation set
val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val, verbose=0)
print(f'Validation Accuracy: {val_accuracy}')

Validation Accuracy: 0.9481893181800842


In [36]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [37]:
# Making predictions on the validation set
preds = model.predict(x_submission_scaled)
# Convert probabilities to binary predictions
binary_preds = (preds > 0.5).astype(int)

# Evaluate the model using the defined function
get_clf_eval(y_submission, binary_preds.flatten())
# Assuming df_test is your test DataFrame and it's already preprocessed similarly to df_train
# Remember to scale the test features as well
# X_test_scaled = scaler.transform(df_test.drop(['is_converted', 'id'], axis=1).values)

# # Making predictions on the test set
# test_preds = model.predict(X_test_scaled)
# # Convert probabilities to binary predictions for the test set
# binary_test_preds = (test_preds > 0.5).astype(int)
# # Summing up the True predictions to get the count
# print(f"Count of Predicted True conversions in Test set: {np.sum(binary_test_preds)}")

165/165 [==============================] - 0s 1ms/step
오차행렬:
 [[  31 1150]
 [ 296 3794]]

정확도: 0.7257
정밀도: 0.0948
재현율: 0.0262
F1: 0.0411


In [38]:
# Predict on the test set
test_pred = model.predict(x_submission_scaled)
test_pred = (test_pred > 0.5).astype(int).reshape(-1)  # Convert probabilities to binary predictions
sum(test_pred) # True로 예측된 개수

165/165 [==============================] - 0s 762us/step


327

In [39]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
# df_sub = pd.read_csv("encoded_submission_two.csv")
# df_sub["is_converted"] = test_pred

# # 제출 파일 저장
# df_sub.to_csv("submission.csv", index=False)